# Simulateur du calcule de lImpot sur le revenu (IR)
Base: openfisca - https://github.com/openfisca

In [1]:
from pprint import pprint  # data pretty printer

from openfisca_core import reforms, periods
from openfisca_france.model.base import *
from openfisca_france.scenarios import init_single_entity

from openfisca_france import FranceTaxBenefitSystem

from openfisca_core.simulation_builder import SimulationBuilder

import numpy as np
import pandas as pd

import plotly.express as px


### Mise à jour des seuils des tranches de l'IR (Projet de loi de finances 2024)

In [2]:
# Cette partie décrit les changements
def modifier_un_parametre(parameters):
    # Ceci décrit la periode sur laquelle va s'appliquer ce changement
    reform_year = 2024
    reform_period = periods.period(reform_year)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[1].threshold.update(start=reform_period.start, value=11520)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[2].threshold.update(start=reform_period.start, value=29373)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[3].threshold.update(start=reform_period.start, value=83988)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[4].threshold.update(start=reform_period.start, value=180648)
    return parameters

# Cette partie rassemble les changements de paramètre dans une seule réforme appelée ici MaReforme
class MaReforme(reforms.Reform):
    def apply(self):
        self.modify_parameters(modifier_function = modifier_un_parametre)


Sanity check et comparaison avec année précédente

In [3]:
legislation_france = FranceTaxBenefitSystem()

resultat_actuel = legislation_france.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024)

print("Résultat actuel")
print(resultat_actuel)

# Consultez la situation avec la reforme
legislation_reforme = MaReforme(legislation_france)

resultat_apres_reforme = legislation_reforme.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024)

print("Resultat après reforme")
print(resultat_apres_reforme)

Résultat actuel
  - threshold: 0
    rate: 0
  - threshold: 11294
    rate: 0.11
  - threshold: 28797
    rate: 0.3
  - threshold: 82341
    rate: 0.41
  - threshold: 177106
    rate: 0.45
Resultat après reforme
  - threshold: 0
    rate: 0
  - threshold: 11520
    rate: 0.11
  - threshold: 29373
    rate: 0.3
  - threshold: 83988
    rate: 0.41
  - threshold: 180648
    rate: 0.45


In [25]:
legislation_reforme.parameters.impot_revenu.calcul_impot_revenu.plaf_qf.plafond_avantages_procures_par_demi_part.celib
legislation_reforme.parameters.impot_revenu.calcul_impot_revenu.plaf_qf.plafond_avantages_procures_par_demi_part.celib.update()


ValueError: You must provide either a start or a period

### Definition du foyer fiscal sur lequel on va appliquer le calcul d'IR

In [4]:
TEST_CASE = {
    'individus': {
        'parent1': {
            'age': {'2015-01': 30},
            'salaire_de_base': {'2023': 100000,'2024': 100000}
        },
        'enfant1': {
            'age': {'2023-01': 12}
        },
        'enfant2': {
            'age': {'2024-01': 18}
        }
    }
}
TEST_CASE['foyers_fiscaux'] = {
    'foyer_fiscal1': {
        'declarants': ['parent1'],
        'personnes_a_charge': ['enfant1', 'enfant2'],
        'celibataire_ou_divorce': {'2023': True, '2024': True},
        'caseT': {'2023': True, '2024': True}
    }
}

TEST_CASE['menages'] = {
    'menage1': {
        'personne_de_reference': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

TEST_CASE['familles'] = {
    'famille1': {
        'parents': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

# display full test case
pprint(TEST_CASE)

{'familles': {'famille1': {'enfants': ['enfant1', 'enfant2'],
                           'parents': ['parent1']}},
 'foyers_fiscaux': {'foyer_fiscal1': {'caseT': {'2023': True, '2024': True},
                                      'celibataire_ou_divorce': {'2023': True,
                                                                 '2024': True},
                                      'declarants': ['parent1'],
                                      'personnes_a_charge': ['enfant1',
                                                             'enfant2']}},
 'individus': {'enfant1': {'age': {'2023-01': 12}},
               'enfant2': {'age': {'2024-01': 18}},
               'parent1': {'age': {'2015-01': 30},
                           'salaire_de_base': {'2023': 100000,
                                               '2024': 100000}}},
 'menages': {'menage1': {'enfants': ['enfant1', 'enfant2'],
                         'personne_de_reference': ['parent1']}}}


In [5]:
simulation_builder = SimulationBuilder()
simulation_2023 = simulation_builder.build_from_entities(legislation_france, TEST_CASE)
simulation_2024 = simulation_builder.build_from_entities(legislation_reforme, TEST_CASE)

print(simulation_2023.calculate('ir_plaf_qf', '2024'))
print(simulation_2023.calculate('ir_plaf_qf', '2023'))
print(simulation_2024.calculate('ir_plaf_qf', '2024'))
print(simulation_2024.calculate('ir_plaf_qf', '2023'))

[9655.527]
[9663.533]
[9521.227]
[9663.533]


### Simulation faisant varier le salaire de base

In [6]:
scenario = init_single_entity(
    legislation_reforme.new_scenario(),
    
    # Axe declaration
    axes = [[
        dict(                       #  in a dictionary
            count = 100,            # 'count' : indicates the number of step
            min = 0,
            max = 100000,
            name = 'salaire_de_base', # the variable you want to make evolve
            ),
        ]],
    
    period = 2024,
    parent1 = dict(
        date_naissance = '1980-01-01',
    )
)

simulation = scenario.new_simulation()

In [7]:
simulation.calculate_add('salaire_de_base', 2024)

array([     0.     ,   1010.10114,   2020.2023 ,   3030.3027 ,
         4040.4045 ,   5050.5054 ,   6060.6055 ,   7070.707  ,
         8080.809  ,   9090.909  ,  10101.011  ,  11111.11   ,
        12121.211  ,  13131.315  ,  14141.414  ,  15151.515  ,
        16161.618  ,  17171.717  ,  18181.818  ,  19191.918  ,
        20202.021  ,  21212.121  ,  22222.22   ,  23232.326  ,
        24242.422  ,  25252.525  ,  26262.63   ,  27272.725  ,
        28282.828  ,  29292.932  ,  30303.03   ,  31313.13   ,
        32323.236  ,  33333.332  ,  34343.434  ,  35353.54   ,
        36363.637  ,  37373.74   ,  38383.836  ,  39393.938  ,
        40404.043  ,  41414.14   ,  42424.242  ,  43434.344  ,
        44444.44   ,  45454.547  ,  46464.652  ,  47474.75   ,
        48484.844  ,  49494.94   ,  50505.05   ,  51515.152  ,
        52525.26   ,  53535.363  ,  54545.45   ,  55555.55   ,
        56565.656  ,  57575.76   ,  58585.863  ,  59595.953  ,
        60606.06   ,  61616.16   ,  62626.26   ,  63636

In [8]:
income_tax = simulation.calculate('ir_plaf_qf', 2024)
TMI = simulation.calculate('ir_tranche', 2024)
gross_wage = simulation.calculate_add('salaire_de_base', 2024)
taxable_income = simulation.calculate_add('salaire_imposable', 2024)

df_results = pd.DataFrame.from_dict({'salaire': gross_wage, 'IR': income_tax, 'TMI': TMI})
tranches_bareme_en_cours = legislation_reforme.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024).rates
df_results['TMI'] = df_results['TMI'].apply(lambda x: tranches_bareme_en_cours[x])
df_results

fig = px.line(df_results, x="salaire", y=["IR"], color='TMI',
              title='Evolution IR en fonction du salaire')
fig.show()

https://openfisca.org/doc/simulate/replicate-simulation-inputs.html#

In [9]:
TEST_CASE_evol = TEST_CASE
TEST_CASE_evol['axes']= [[{'count':100, 'name':'salaire_de_base', 'min':0, 'max':100000, 'period':'2023'},
                          {'count':100, 'name':'salaire_de_base', 'min':0, 'max':100000, 'period':'2024'}]]
TEST_CASE_evol['individus']['parent1']['f6rs'] = {'2023': 10000, '2024': 5000}
#pprint(TEST_CASE_evol)

simulation_builder = SimulationBuilder()
simulation_evol = simulation_builder.build_from_entities(legislation_reforme, TEST_CASE_evol)
sdb = simulation_evol.calculate_add('salaire_de_base', '2023')
salaire_foyer = sdb.reshape(100,3).sum(1)

simulation_evol.trace = True
income_tax_2023 = simulation_evol.calculate('ir_plaf_qf', 2023)
income_tax_2024 = simulation_evol.calculate('ir_plaf_qf', 2024)
df_income_tax = pd.DataFrame.from_dict({'revenu': salaire_foyer, 'IR_2023': income_tax_2023, 'IR_2024': income_tax_2024})
fig = px.line(df_income_tax, x='revenu', y=['IR_2023', 'IR_2024'])
fig.show()
df_income_tax



,revenu,IR_2023,IR_2024
0,0.000000,0.000000,0.000000
1,1010.101135,0.000000,0.000000
2,2020.202271,0.000000,0.000000
3,3030.302734,0.000000,0.000000
4,4040.404541,0.000000,0.000000
...,...,...,...
95,95959.593750,5757.542969,7115.236328
96,96969.687500,5984.089844,7341.785156
97,97979.804688,6210.568359,7568.266602
98,98989.882812,6437.016602,7794.710938


In [10]:
TEST_CASE_PER = TEST_CASE
TEST_CASE_PER['axes']= [[{'count':200, 'name':'f6rs', 'min':0, 'max':50000, 'period':'2023'},
                          {'count':200, 'name':'f6rs', 'min':0, 'max':50000, 'period':'2024'}]]
#TEST_CASE_evol['individus']['parent1']['f6rs'] = {'2023': 10000, '2024': 5000}
simulation_builder = SimulationBuilder()
simulation_per = simulation_builder.build_from_entities(legislation_reforme, TEST_CASE_PER)
sdb = simulation_per.calculate_add('f6rs', '2023')
versement_PER = sdb.reshape(200,3).sum(1)

income_tax_2023 = simulation_per.calculate('ir_plaf_qf', 2023)
income_tax_2024 = simulation_per.calculate('ir_plaf_qf', 2024)
tmi_2024 = simulation_per.calculate('ir_tranche', 2024)
df_income_tax = pd.DataFrame.from_dict({'Versement_PER': versement_PER, 'IR_2023': income_tax_2023, 'IR_2024': income_tax_2024})
fig = px.line(df_income_tax, x='Versement_PER', y=['IR_2023', 'IR_2024'])
fig.show()
df_tmi = pd.DataFrame.from_dict({'Versement_PER':versement_PER, 'tmi': tmi_2024})
tranches_bareme_en_cours = legislation_reforme.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024).rates
df_tmi['tmi'] = df_tmi['tmi'].apply(lambda x: tranches_bareme_en_cours[x])

fig = px.line(df_tmi, x='Versement_PER', y='tmi')
fig.show()


In [11]:
dir(legislation_reforme.parameters.impot_revenu.calcul_impot_revenu.plaf_qf)
dir(legislation_reforme.entities)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']